In [16]:
from collections import Counter

In [17]:
def parse_input(fl) -> list[dict]:
    hands = []
    with open(fl) as infile:
        for ln in infile.readlines():
            cards, bid = ln.strip().split()
            hands.append({'cards': cards, 'bid': int(bid)})
    return hands

In [18]:
test = parse_input("data/day7-test.txt")
inputs = parse_input("data/day7-input.txt")
test

[{'cards': '32T3K', 'bid': 765},
 {'cards': 'T55J5', 'bid': 684},
 {'cards': 'KK677', 'bid': 28},
 {'cards': 'KTJJT', 'bid': 220},
 {'cards': 'QQQJA', 'bid': 483}]

### Part 1

In [99]:
LBL_RANK_LIST1 = list(reversed(['A', 'K', 'Q', 'J', 'T', '9', '8', '7', '6', '5', '4', '3', '2']))
LBL2SCORE1 = {
    lbl: i
    for i, lbl in enumerate(LBL_RANK_LIST1, start=1)
}
LBL_RANK_LIST2 = list(reversed(['A', 'K', 'Q', 'T', '9', '8', '7', '6', '5', '4', '3', '2', 'J']))
LBL2SCORE2 = {
    lbl: i
    for i, lbl in enumerate(LBL_RANK_LIST2, start=1)
}
print(LBL2SCORE1)
print(LBL2SCORE2)

{'2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, 'T': 9, 'J': 10, 'Q': 11, 'K': 12, 'A': 13}
{'J': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'T': 10, 'Q': 11, 'K': 12, 'A': 13}


In [106]:
import functools as ft

def part1(hands):
    return part_helper(
        hands=hands,
        _pair_score_fn=_pair_score_part1,
        lbl2score=LBL2SCORE2,
    )

def part2(hands):
    return part_helper(
        hands=hands,
        _pair_score_fn=_pair_score_part2,
        lbl2score=LBL2SCORE2,
    )

def part_helper(hands, _pair_score_fn, lbl2score):
    ranked_hands = sorted(
        hands,
        key=ft.cmp_to_key(
            ft.partial(
                cmp_poker,
                _pair_score_fn=_pair_score_fn,
                lbl2score=lbl2score,
            ),
        ),
    )
    return sum(h['bid'] * rk for rk, h in enumerate(ranked_hands, start=1))
    
def cmp_poker(hand1, hand2, _pair_score_fn, lbl2score):
    cards1, cards2 = hand1['cards'], hand2['cards']
    ps_c1, ps_c2 = _pair_score_fn(cards1), _pair_score_fn(cards2)
    if ps_c1 != ps_c2:
        return ps_c1-ps_c2
    # compare cards in order
    for c1, c2 in zip(cards1, cards2):
        if c1 == c2:
            continue
        # print(f'{c1=},{c2=}')
        return lbl2score[c1] - lbl2score[c2]
    else:
        return 0

def _pair_score_part1(cards):
    return _to_pair_score(cntr=Counter(cards))

def _to_pair_score(cntr):
    return sum(20**ct for ct in cntr.values())

def _pair_score_part2(cards):
    cntr = Counter(cards)
    if 'J' not in cntr:
        return  _to_pair_score(cntr=cntr)
    if 'J' in cntr and len(cntr) == 1:
        return  _to_pair_score(cntr=cntr)
    j_cnt = cntr.pop('J')
    rem_max_cnt = max(cntr.values())
    for c in cntr:
        if cntr[c] == rem_max_cnt:
            cntr[c] += j_cnt
            # print(f"{cards=}, {cntr=}")
            break
    return _to_pair_score(cntr=cntr)

In [107]:
print(part1(test))
print(part1(inputs))

6440
250257348


In [108]:
cmp_poker(
    hand1={'cards': 'KTJJT'},
    hand2={'cards': 'QQQJA'},
    _pair_score_fn=_pair_score_part2,
    lbl2score=LB
)

TypeError: cmp_poker() missing 1 required positional argument: 'lbl2score'

In [98]:
print(part2(test))
print(part2(inputs))

5905
248846325
